In [1]:
!pip -q install transformers sentencepiece datasets torch evaluate seqeval underthesea sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   

In [2]:
import pandas as pd
import numpy as np
import re
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
import evaluate
import os
from tqdm.auto import tqdm
from typing import List, Dict, Tuple
import json

2025-11-22 09:21:30.733882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763803290.922622      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763803290.975081      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"GPU is ready: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is ready: Tesla P100-PCIE-16GB


# 1. Configuration

In [4]:
MODEL_NAME = "5CD-AI/visobert-14gb-corpus"
MAX_LENGTH = 512
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 128
GRADIENT_ACCUMULATION = 1
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 0.01
GRAD_NORM = 1.0
NUM_EPOCHS = 15
WARMUP_RATIO = 0.1
LABEL_SMOOTHING_FACTOR = 0.1
OUTPUT_DIR = "./visobert_model"

In [5]:
# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
from underthesea import sent_tokenize

def smart_chunk_text(item: dict, max_length: int = 512) -> List[dict]:
    """
    Chia nhỏ văn bản thành các chunk và tính toán lại nhãn dựa trên lát cắt thực tế.
    """
    text = item['text']
    original_labels = item.get('label', [])
    
    # 1. Tách câu
    sentences = sent_tokenize(text)
    
    chunks = []
    
    # Biến lưu trữ các câu trong chunk hiện tại
    current_chunk_sentences = [] # List các tuple (start, end, text)
    current_word_count = 0
    
    # Con trỏ tìm kiếm trong văn bản gốc
    search_cursor = 0
    
    # Hàm helper để tạo chunk từ danh sách các câu đã gom
    def flush_chunk(sent_buffer):
        if not sent_buffer:
            return
            
        # Lấy vị trí bắt đầu của câu đầu tiên và kết thúc của câu cuối cùng
        chunk_start = sent_buffer[0][0]
        chunk_end = sent_buffer[-1][1]
        
        # Cắt văn bản gốc để đảm bảo giữ nguyên mọi khoảng trắng/xuống dòng gốc
        chunk_text = text[chunk_start:chunk_end]
        
        new_labels = []
        for lbl in original_labels:
            # Kiểm tra nếu label nằm trọn vẹn trong chunk này
            if lbl['start'] >= chunk_start and lbl['end'] <= chunk_end:
                new_lbl = lbl.copy()
                # Tính lại toạ độ tương đối so với chunk mới
                new_lbl['start'] = lbl['start'] - chunk_start
                new_lbl['end'] = lbl['end'] - chunk_start
                new_labels.append(new_lbl)
        
        chunks.append({
            "text": chunk_text, 
            "label": new_labels
        })

    for sent in sentences:
        # Tìm vị trí chính xác của câu trong văn bản gốc
        sent_start = text.find(sent, search_cursor)
        if sent_start == -1: 
            # Fallback nếu không tìm thấy (hiếm gặp), bỏ qua hoặc dùng cursor hiện tại
            sent_start = search_cursor
            
        sent_end = sent_start + len(sent)
        sent_word_count = len(sent.split())
        
        # Kiểm tra nếu thêm câu này vào thì có vượt quá max_length không
        if current_word_count + sent_word_count <= max_length:
            current_chunk_sentences.append((sent_start, sent_end, sent))
            current_word_count += sent_word_count
        else:
            # Nếu vượt quá, đóng gói chunk cũ
            flush_chunk(current_chunk_sentences)
            
            # Tạo chunk mới bắt đầu bằng câu hiện tại
            current_chunk_sentences = [(sent_start, sent_end, sent)]
            current_word_count = sent_word_count
            
        # Cập nhật con trỏ tìm kiếm
        search_cursor = sent_end

    # Đóng gói chunk cuối cùng nếu còn
    if current_chunk_sentences:
        flush_chunk(current_chunk_sentences)
            
    return chunks

In [7]:
# Tagged label
LABELS = ["PER", "ORG", "ADDR"]

# NER labels with BIO format
NER_LABELS = ["O"] + [f"{prefix}-{label}" for prefix in ["B", "I"] for label in LABELS]
ID2LABEL = {i: label for i, label in enumerate(NER_LABELS)}
LABEL2ID = {label: i for i, label in enumerate(NER_LABELS)}

# 2. prepare dataset

In [8]:
def create_bio_labels_from_spans(text, spans):
    """
    Creates character-level BIO labels from spans with stricter validation
    
    Args:
        text: Clean text without tags
        spans: List of entity spans with start, end, text, labels
    
    Returns:
        List of character-level BIO labels
    """
    # Initialize all characters with O label
    bio_labels = ["O"] * len(text)
    
    # Sort spans by start position to ensure proper BIO order
    sorted_spans = sorted(spans, key=lambda x: x["start"])
    
    # Add entity labels
    for span in sorted_spans:
        start = span["start"]
        end = span["end"]
        label = span["labels"][0]
        
        # Check if indices are valid
        if start < 0 or end > len(text) or start >= len(text) or end <= 0:
            print(f"Warning: Invalid span indices ({start}, {end}) for text of length {len(text)}")
            continue
        
        # Check for overlapping spans
        if any(l != "O" for l in bio_labels[start:end]):
            print(f"Warning: Overlapping span at position ({start}, {end})")
            # Handle overlapping spans - prioritize this span
            # You could also choose to skip or merge spans based on your requirements
        
        # Set beginning token
        bio_labels[start] = f"B-{label}"
        
        # Set inside tokens
        for i in range(start + 1, end):
            bio_labels[i] = f"I-{label}"
    
    # Validate for any inconsistencies
    for i in range(1, len(bio_labels)):
        if bio_labels[i].startswith("I-"):
            entity_type = bio_labels[i][2:]
            if not (bio_labels[i-1].startswith("B-" + entity_type) or 
                   bio_labels[i-1].startswith("I-" + entity_type)):
                print(f"Warning: Invalid BIO sequence at position {i}: '{bio_labels[i-1]}' followed by '{bio_labels[i]}'")
                # Auto-correct: Convert invalid I- tag to B-
                bio_labels[i] = f"B-{entity_type}"
    
    return bio_labels

In [9]:
def convert_tagged_data_to_token_classification(item: dict) -> Tuple[str, List[str]]:
    """
    Convert text with tags to token classification format
    
    Args:
        tagged_text: Text with entity tags like <PER>Name</PER>
        
    Returns:
        Tuple of (text without tags, list of token labels)
    """
    clean_text = item['text']
    spans = item.get('label', [])
    
    # Create BIO labels from spans
    char_labels = create_bio_labels_from_spans(clean_text, spans)
    
    # Double check lengths match
    if len(clean_text) != len(char_labels):
        print(f"Warning: Mismatch between text length ({len(clean_text)}) and labels length ({len(char_labels)})")
        # Truncate the longer one to match
        min_len = min(len(clean_text), len(char_labels))
        clean_text = clean_text[:min_len]
        char_labels = char_labels[:min_len]
    
    return clean_text, char_labels

In [10]:
def tokenize_and_align_labels(examples):
    """
    Tokenize text and align character-level labels to token-level labels
    with improved BIO consistency
    """
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenized_inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        return_offsets_mapping=True,
    )
    
    labels = []
    
    for i, (text, char_labels) in enumerate(zip(examples["text"], examples["char_labels"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        previous_bio_tag = "O"
        
        for word_idx, offsets in zip(word_ids, tokenized_inputs["offset_mapping"][i]):
            # Special tokens have word_idx set to None
            if word_idx is None:
                label_ids.append(-100)
                continue
                
            # Empty token (e.g., special token)
            if offsets[0] == offsets[1]:
                label_ids.append(-100)
                continue
                
            # Handle case where offsets are out of bounds
            if offsets[0] >= len(char_labels) or offsets[1] > len(char_labels):
                label_ids.append(-100)
                continue
                
            # For normal tokens, get the character labels within token span
            start, end = offsets
            span_labels = char_labels[start:end]
            
            if not span_labels:
                label_ids.append(LABEL2ID["O"])
                previous_bio_tag = "O"
                continue
                
            # Count labels in span
            from collections import Counter
            label_counts = Counter(span_labels)
            
            # Remove "O" if mixed with other labels
            if len(label_counts) > 1 and "O" in label_counts:
                del label_counts["O"]
            
            # Always prefer B- label if exists in the span (to ensure BIO validity)
            bio_candidates = [label for label in span_labels if label.startswith("B-")]
            if bio_candidates:
                most_common_label = bio_candidates[0]
            else:
                most_common_label = label_counts.most_common(1)[0][0]
            
            # Apply BIO consistency rules
            if most_common_label.startswith("I-"):
                entity_type = most_common_label[2:]
                
                # If previous token was of different entity or was O, convert to B-
                if previous_bio_tag == "O" or (
                    previous_bio_tag.startswith("B-") or previous_bio_tag.startswith("I-")
                ) and previous_bio_tag[2:] != entity_type:
                    most_common_label = "B-" + entity_type
                    
                # If same word but different entity than previous token
                elif word_idx == previous_word_idx and previous_bio_tag[2:] != entity_type:
                    most_common_label = "B-" + entity_type
            
            label_ids.append(LABEL2ID[most_common_label])
            previous_bio_tag = most_common_label
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [11]:
def prepare_dataset_with_chunking(train_list, val_list, test_list):
    """
    Prepare datasets with smart chunking from a list of dicts (JSON)
    """
    def process_list(data_list, desc):
        processed_data = []
        for item in tqdm(data_list, desc=desc):
            try:
                # Chunking the text
                # smart_chunk_text now expects a dict item
                chunks = smart_chunk_text(item)
                
                for chunk in chunks:
                    try:
                        # convert now expects a dict chunk
                        text, char_labels = convert_tagged_data_to_token_classification(chunk)
                        if text.strip():  # Only add if content exists
                            processed_data.append({"text": text, "char_labels": char_labels})
                    except Exception as e:
                        print(f"Error processing chunk: {e}")
                        continue
            except Exception as e:
                # Use .get('id') if available for debugging
                print(f"Error processing item {item.get('id', 'unknown')}: {e}")
                continue
        
        return processed_data
    
    # Process data
    train_data = process_list(train_list, "Processing train data")
    val_data = process_list(val_list, "Processing validation data")
    test_data = process_list(test_list, "Processing test data")
    
    print(f"Original train samples: {len(train_list)}, After chunking: {len(train_data)}")
    print(f"Original val samples: {len(val_list)}, After chunking: {len(val_data)}")
    print(f"Original test samples: {len(test_list)}, After chunking: {len(test_data)}")
    
    # Convert to HuggingFace datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)
    
    # Combine into a DatasetDict
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    
    return dataset_dict

# 3. Setup metric, postprocessing

In [12]:
# def fix_bio_tags(tags):
#     """
#     Fix invalid BIO tag sequences by converting first I- tags without preceding B- to B- tags
#     """
#     fixed_tags = list(tags)
    
#     for i in range(len(fixed_tags)):
#         if fixed_tags[i].startswith('I-'):
#             entity_type = fixed_tags[i][2:]
            
#             if i == 0 or (not fixed_tags[i-1].startswith('B-' + entity_type) and 
#                           not fixed_tags[i-1].startswith('I-' + entity_type)):
#                 fixed_tags[i] = 'B-' + entity_type
    
#     return fixed_tags
def fix_bio_tags(tags):
    """
    Fix invalid BIO tag sequences with multiple rules:
    1. If an entity token is surrounded by O tags on both sides, convert it to O
    2. If an O token is between the same entity type on both sides, convert it to I-entity
    3. Convert I- tags without preceding B-/I- of same type to B- tags
    """
    fixed_tags = list(tags)
    
    # Rule 1: Entity token surrounded by O on BOTH sides -> convert to O
    for i in range(len(fixed_tags)):
        if fixed_tags[i].startswith(('B-', 'I-')):
            left_is_o = (i == 0) or (fixed_tags[i-1] == 'O')
            right_is_o = (i == len(fixed_tags) - 1) or (fixed_tags[i+1] == 'O')
            
            if left_is_o and right_is_o:
                fixed_tags[i] = 'O'
    
    # Rule 2: O token between same entity type -> convert to I-entity
    # Apply multiple times to fill consecutive O gaps
    changed = True
    while changed:
        changed = False
        for i in range(1, len(fixed_tags) - 1):
            if fixed_tags[i] == 'O':
                left_tag = fixed_tags[i-1]
                right_tag = fixed_tags[i+1]
                
                # Check if both neighbors are entity tags
                if left_tag.startswith(('B-', 'I-')) and right_tag.startswith(('B-', 'I-')):
                    left_entity = left_tag[2:]
                    right_entity = right_tag[2:]
                    
                    # If same entity type, fill the gap
                    if left_entity == right_entity:
                        fixed_tags[i] = f'I-{left_entity}'
                        changed = True
    
    # Rule 3: Fix I- tags without proper preceding tag (original logic)
    for i in range(len(fixed_tags)):
        if fixed_tags[i].startswith('I-'):
            entity_type = fixed_tags[i][2:]
            
            if i == 0 or (not fixed_tags[i-1].startswith('B-' + entity_type) and 
                          not fixed_tags[i-1].startswith('I-' + entity_type)):
                fixed_tags[i] = 'B-' + entity_type
    
    return fixed_tags

In [13]:
def compute_metrics(p):
    """
    Compute NER metrics using seqeval with detailed per-label results
    """
    seqeval = evaluate.load("seqeval")
    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # Remove ignored index (-100)
    true_predictions = [
        [ID2LABEL[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_predictions = [fix_bio_tags(preds) for preds in true_predictions]
    
    true_labels = [
        [ID2LABEL[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [fix_bio_tags(labels) for labels in true_labels]
    
    # Compute overall metrics
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    
    # Compute per-label metrics
    per_label_results = {}
    for label in LABELS:  # ["PER", "ORG", "ADDR"]
        # Filter predictions and labels for current entity type
        label_predictions = []
        label_references = []
        
        for pred_seq, ref_seq in zip(true_predictions, true_labels):
            pred_filtered = ["O" if not tag.endswith(f"-{label}") else tag for tag in pred_seq]
            ref_filtered = ["O" if not tag.endswith(f"-{label}") else tag for tag in ref_seq]
            
            label_predictions.append(pred_filtered)
            label_references.append(ref_filtered)
        
        # Compute metrics for this label
        try:
            label_metrics = seqeval.compute(predictions=label_predictions, references=label_references)
            per_label_results[f"{label}_precision"] = label_metrics["overall_precision"]
            per_label_results[f"{label}_recall"] = label_metrics["overall_recall"]
            per_label_results[f"{label}_f1"] = label_metrics["overall_f1"]
            per_label_results[f"{label}_accuracy"] = label_metrics["overall_accuracy"]
        except:
            # Handle case where no entities of this type exist
            per_label_results[f"{label}_precision"] = 0.0
            per_label_results[f"{label}_recall"] = 0.0
            per_label_results[f"{label}_f1"] = 0.0
            per_label_results[f"{label}_accuracy"] = 0.0
    
    # Combine overall and per-label results
    combined_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        **per_label_results
    }
    
    return combined_results

# 4. Train & Evaluate

In [14]:
!gdown 1wN3xamHTh2XDzLyn5KWJFC8uJH9MaYy3

Downloading...
From: https://drive.google.com/uc?id=1wN3xamHTh2XDzLyn5KWJFC8uJH9MaYy3
To: /kaggle/working/data_labeled.json
100%|██████████████████████████████████████| 26.3M/26.3M [00:00<00:00, 50.6MB/s]


In [15]:
import json
from sklearn.model_selection import train_test_split

def load_data():
    """
    Load data from single Json file and split into train, validation, and test sets
    80-10-10 split
    """
    # Load the single data file
    data_path = '/kaggle/working/data_labeled.json'

    # Load JSON
    with open(data_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)    
    
    # First split: 80% train, 20% temp (val + test)
    train_df, temp_df = train_test_split(
        raw_data, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True
    )
    
    # Second split: from the 20%, split into 50% val and 50% test (10% each of total)
    val_df, test_df = train_test_split(
        temp_df, 
        test_size=0.5,  # 0.5 * 0.2 = 0.1 (10% of total for test)
        random_state=42, 
        shuffle=True
    )
    
    print(f"Data split completed:")
    print(f"  Train set: {len(train_df)} samples ({len(train_df)/len(raw_data)*100:.1f}%)")
    print(f"  Validation set: {len(val_df)} samples ({len(val_df)/len(raw_data)*100:.1f}%)")
    print(f"  Test set: {len(test_df)} samples ({len(test_df)/len(raw_data)*100:.1f}%)")
    
    return train_df, val_df, test_df

In [16]:
# Prepare datasets với chunking thông minh
print("Preparing datasets với smart chunking...")
train_df, val_df, test_df = load_data()
dataset_dict = prepare_dataset_with_chunking(train_df, val_df, test_df)

Preparing datasets với smart chunking...
Data split completed:
  Train set: 2010 samples (80.0%)
  Validation set: 251 samples (10.0%)
  Test set: 252 samples (10.0%)


Processing train data:   0%|          | 0/2010 [00:00<?, ?it/s]

Processing validation data:   0%|          | 0/251 [00:00<?, ?it/s]

Processing test data:   0%|          | 0/252 [00:00<?, ?it/s]

Original train samples: 2010, After chunking: 5821
Original val samples: 251, After chunking: 767
Original test samples: 252, After chunking: 808


In [17]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.model_max_length = MAX_LENGTH

# Tokenize and align labels
print("Tokenizing and aligning labels...")
tokenized_datasets = dataset_dict.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=["text", "char_labels"]
    )

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Tokenizing and aligning labels...


Map:   0%|          | 0/5821 [00:00<?, ? examples/s]

Map:   0%|          | 0/767 [00:00<?, ? examples/s]

Map:   0%|          | 0/808 [00:00<?, ? examples/s]

In [18]:
def freeze_half_model_layers(model):
    """
    Freezes half of the encoder layers in a DeBERTa model.
    Assumes the model follows the standard DeBERTa architecture.
    
    Args:
        model: A DeBERTaV2 model (e.g., DebertaV2ForTokenClassification)
    
    Returns:
        The model with half of its encoder layers frozen
    """
    # First, identify the total number of layers
    if hasattr(model, "roberta") and hasattr(model.roberta, "encoder") and hasattr(model.roberta.encoder, "layer"):
        num_layers = len(model.roberta.encoder.layer)
        print(f"Model has {num_layers} transformer layers")
        
        # Calculate which layers to freeze
        layers_to_freeze = num_layers // 4
        
        # Freeze the first half of the layers
        for i in range(layers_to_freeze):
            for param in model.roberta.encoder.layer[i].parameters():
                param.requires_grad = False
            print(f"Layer {i} frozen")
        
        # Keep the second half trainable
        for i in range(layers_to_freeze, num_layers):
            print(f"Layer {i} remains trainable")
        
        # You can also freeze embeddings if desired
        # for param in model.deberta.embeddings.parameters():
        #     param.requires_grad = False
        # print("Embeddings frozen")
        
        # Count trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.1%} of total)")
        
        return model
    else:
        raise ValueError("Model structure doesn't match expected DeBERTa architecture")

In [19]:
# Load model
print("Loading model...")
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(NER_LABELS),
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

# model = freeze_half_model_layers(model)

Loading model...


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at 5CD-AI/visobert-14gb-corpus and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Data collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=MAX_LENGTH,
    pad_to_multiple_of=16
)

In [21]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    weight_decay=WEIGHT_DECAY,
    max_grad_norm=GRAD_NORM,
    save_total_limit=2,
    num_train_epochs=NUM_EPOCHS,
    fp16=True,
    fp16_opt_level="O1",
    warmup_ratio=WARMUP_RATIO,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    label_smoothing_factor=LABEL_SMOOTHING_FACTOR,
    push_to_hub=False,
    report_to=[],
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    remove_unused_columns=True,
    group_by_length=True,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=3,
    )]
)

/tmp/ipykernel_19/2265847917.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# Train model
print("Starting training...")
trainer.train()

# Save model
trainer.save_model(f"{OUTPUT_DIR}/final")
print(f"Model saved to {OUTPUT_DIR}/final")

Starting training...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Per Precision,Per Recall,Per F1,Per Accuracy,Org Precision,Org Recall,Org F1,Org Accuracy,Addr Precision,Addr Recall,Addr F1,Addr Accuracy
1,1.216000,0.499204,0.274965,0.285164,0.279972,0.977631,0.000000,0.000000,0.000000,0.993817,0.289730,0.375186,0.326967,0.984058,0.274448,0.251445,0.262443,0.996143
2,0.487700,0.471373,0.611416,0.696532,0.651205,0.989651,0.457557,0.575521,0.509804,0.997184,0.655016,0.751857,0.700104,0.993094,0.632047,0.615607,0.623719,0.998202
3,0.471900,0.468579,0.691878,0.775530,0.731320,0.990575,0.565315,0.653646,0.606280,0.997291,0.720107,0.800892,0.758354,0.993780,0.727979,0.812139,0.767760,0.998295
4,0.467300,0.467145,0.722079,0.809730,0.763397,0.991111,0.607623,0.705729,0.653012,0.997396,0.743280,0.821694,0.780522,0.993999,0.771574,0.878613,0.821622,0.998429
5,0.465800,0.466145,0.744074,0.816474,0.778594,0.991543,0.638095,0.697917,0.666667,0.997388,0.760870,0.832095,0.794890,0.994313,0.795337,0.887283,0.838798,0.998565
6,0.464000,0.465411,0.763883,0.815029,0.788627,0.991889,0.653488,0.731771,0.690418,0.997371,0.782578,0.820951,0.801305,0.994606,0.820375,0.884393,0.851182,0.998605
7,0.462900,0.465795,0.769895,0.815511,0.792047,0.991835,0.658768,0.723958,0.689826,0.997379,0.787385,0.825409,0.805948,0.994569,0.830601,0.878613,0.853933,0.998565
8,0.462000,0.465662,0.773333,0.810212,0.791343,0.992070,0.664234,0.710938,0.686792,0.997374,0.791547,0.820951,0.805981,0.994719,0.826087,0.878613,0.851541,0.998616
9,0.460400,0.465992,0.773772,0.827071,0.799534,0.991721,0.677184,0.726562,0.701005,0.997419,0.780860,0.836553,0.807747,0.994260,0.854795,0.901734,0.877637,0.998698
10,0.460100,0.466151,0.781307,0.829480,0.804673,0.992039,0.678571,0.742188,0.708955,0.997433,0.797040,0.840267,0.818083,0.994637,0.838356,0.884393,0.860759,0.998616


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Model saved to ./visobert_model/final


In [23]:
# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(tokenized_datasets["test"], metric_key_prefix="test")

# Display results in organized format
print("\n" + "="*60)
print("📊 DETAILED TEST RESULTS")
print("="*60)

# Overall metrics
print("\n🔍 OVERALL METRICS:")
print(f"  Precision: {test_results.get('test_precision', 0):.4f}")
print(f"  Recall:    {test_results.get('test_recall', 0):.4f}")
print(f"  F1-Score:  {test_results.get('test_f1', 0):.4f}")
print(f"  Accuracy:  {test_results.get('test_accuracy', 0):.4f}")

# Per-label metrics
print("\n📈 PER-LABEL METRICS:")
for label in LABELS:
    print(f"\n  {label} Entity:")
    print(f"    Precision: {test_results.get(f'test_{label}_precision', 0):.4f}")
    print(f"    Recall:    {test_results.get(f'test_{label}_recall', 0):.4f}")
    print(f"    F1-Score:  {test_results.get(f'test_{label}_f1', 0):.4f}")
    print(f"    Accuracy:  {test_results.get(f'test_{label}_accuracy', 0):.4f}")

print("\n" + "="*60)

# Save detailed test results
with open(f"{OUTPUT_DIR}/detailed_test_results.txt", "w", encoding="utf-8") as f:
    f.write("DETAILED NER MODEL TEST RESULTS\n")
    f.write("="*50 + "\n\n")
    
    f.write("OVERALL METRICS:\n")
    f.write(f"Precision: {test_results.get('test_precision', 0):.4f}\n")
    f.write(f"Recall: {test_results.get('test_recall', 0):.4f}\n")
    f.write(f"F1-Score: {test_results.get('test_f1', 0):.4f}\n")
    f.write(f"Accuracy: {test_results.get('test_accuracy', 0):.4f}\n\n")
    
    f.write("PER-LABEL METRICS:\n")
    for label in LABELS:
        f.write(f"\n{label} Entity:\n")
        f.write(f"  Precision: {test_results.get(f'test_{label}_precision', 0):.4f}\n")
        f.write(f"  Recall: {test_results.get(f'test_{label}_recall', 0):.4f}\n")
        f.write(f"  F1-Score: {test_results.get(f'test_{label}_f1', 0):.4f}\n")
        f.write(f"  Accuracy: {test_results.get(f'test_{label}_accuracy', 0):.4f}\n")
    
    f.write(f"\nFull results dictionary:\n")
    for key, value in test_results.items():
        f.write(f"{key}: {value}\n")

print(f"📁 Detailed test results saved to {OUTPUT_DIR}/detailed_test_results.txt")

Evaluating on test set...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


early stopping required metric_for_best_model, but did not find eval_f1 so early stopping is disabled



📊 DETAILED TEST RESULTS

🔍 OVERALL METRICS:
  Precision: 0.7951
  Recall:    0.8671
  F1-Score:  0.8295
  Accuracy:  0.9934

📈 PER-LABEL METRICS:

  PER Entity:
    Precision: 0.6558
    Recall:    0.8145
    F1-Score:  0.7266
    Accuracy:  0.9986

  ORG Entity:
    Precision: 0.8329
    Recall:    0.8857
    F1-Score:  0.8585
    Accuracy:  0.9962

  ADDR Entity:
    Precision: 0.7336
    Recall:    0.8109
    F1-Score:  0.7703
    Accuracy:  0.9982

📁 Detailed test results saved to ./visobert_model/detailed_test_results.txt
